In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.data import AUTOTUNE
from frontier_stitching import gen_adversaries, verify

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
if len(physical_devices) > 0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [3]:
def to_float(x, y):
    return tf.cast(x, tf.float32) / 255.0, y

def comp(model):
    model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), 
              metrics=["sparse_categorical_accuracy"])

dataset = tfds.load("mnist", split="train", as_supervised=True)
val_set = tfds.load("mnist", split="test", as_supervised=True)

dataset = dataset.map(to_float).shuffle(1024).batch(128).prefetch(AUTOTUNE)
val_set = val_set.map(to_float).batch(128)

model = keras.Sequential([
            keras.layers.Conv2D(16, 3, padding="same", activation="relu"),
            keras.layers.Conv2D(32, 3, padding="same", strides=2, activation="relu"),
            keras.layers.Conv2D(64, 3, padding="same", strides=2, activation="relu"),
            keras.layers.Flatten(),
            keras.layers.Dense(10, activation=None)
        ])

comp(model)
model.build(input_shape=(None, 28, 28, 1))

# pretrain the model
model.fit(dataset, epochs=3, validation_data=val_set)

l = 100

# generate key set
true_advs, false_advs = gen_adversaries(model, l, dataset, 0.1)

# In case that not the full number of adversaries could be generated a reduced amount is returned
assert(len(true_advs + false_advs) == l)

key_set_x = tf.data.Dataset.from_tensor_slices([x for x, y in true_advs + false_advs])
key_set_y = tf.data.Dataset.from_tensor_slices([y for x, y in true_advs + false_advs])
key_set = tf.data.Dataset.zip((key_set_x, key_set_y)).batch(128)

Epoch 1/3
469/469 [==============================] - 5s 7ms/step - loss: 0.9382 - sparse_categorical_accuracy: 0.6999 - val_loss: 0.1161 - val_sparse_categorical_accuracy: 0.9665
Epoch 2/3
469/469 [==============================] - 2s 5ms/step - loss: 0.1205 - sparse_categorical_accuracy: 0.9638 - val_loss: 0.0910 - val_sparse_categorical_accuracy: 0.9709
Epoch 3/3
469/469 [==============================] - 2s 5ms/step - loss: 0.0863 - sparse_categorical_accuracy: 0.9740 - val_loss: 0.0714 - val_sparse_categorical_accuracy: 0.9782


In [4]:
# reset the optimizer and embed the watermark
comp(model)
model.fit(key_set, epochs=5, validation_data=val_set)

Epoch 1/5
1/1 [==============================] - 1s 810ms/step - loss: 2.0916 - sparse_categorical_accuracy: 0.5000 - val_loss: 0.0658 - val_sparse_categorical_accuracy: 0.9790
Epoch 2/5
1/1 [==============================] - 0s 454ms/step - loss: 1.5772 - sparse_categorical_accuracy: 0.5500 - val_loss: 0.0688 - val_sparse_categorical_accuracy: 0.9779
Epoch 3/5
1/1 [==============================] - 0s 452ms/step - loss: 0.9644 - sparse_categorical_accuracy: 0.6600 - val_loss: 0.0825 - val_sparse_categorical_accuracy: 0.9757
Epoch 4/5
1/1 [==============================] - 0s 480ms/step - loss: 0.5848 - sparse_categorical_accuracy: 0.8200 - val_loss: 0.0986 - val_sparse_categorical_accuracy: 0.9686
Epoch 5/5
1/1 [==============================] - 0s 488ms/step - loss: 0.3468 - sparse_categorical_accuracy: 0.8800 - val_loss: 0.1165 - val_sparse_categorical_accuracy: 0.9633


In [5]:
info = verify(model, key_set, 0.05)

In [6]:
if info["success"]:
    print("Model is ours and was successfully watermarked.")
else:
    print("Model is not ours and was not successfully watermarked.")

Model is ours and was successfully watermarked.
